# HW3 Corpus Analysis

Dataset 1: (Topic: Restaurants and restaurant reviews)
https://drive.google.com/file/d/11H6AbWxKsPLY3yt__OrmK0rjjYShKhig/view?usp=sharing

Dataset 2: (Topic: Movie reviews)
https://drive.google.com/file/d/17nGHPsk4RXfvRoq-ndizTzc0_0PkzAqm/view?usp=sharing

Both of these datasets are csv files. However, Dataset 1 is easier to clean and prepare. Dataset 2 is more of a challenge.

Your goal for both is to use Python and CountVectorizer and to prepare and format the datasets into dataframe. The first column in each dataframe should be called “LABEL” and will be the label of the data for that row. 

*** Because I am giving you two datasets to prepare, I am WAIVING the FORMAT – this time only. You DO NOT need to have an Intro or a Conclusions. 

In Analysis you will talk about the two datasets, how they are similar, how they are different, and the challenges you faced in preparing and formatting each into labeled data frames. 

In Results you will include images of each of the dataframes (partial because they are huge) for each data set. 

Show the BEFORE (the raw data) and the AFTER (the clean and labeled dataframe). Do this for each dataset. 

Keep in mind that because the dataset and dataframe are BIG – you will only need to create images that show portions of each. 


In [1]:
import pandas as pd
import re

In [2]:
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_colwidth', 1000)

## Dataset 1

In [3]:
restaurant = pd.read_csv('RestaurantSentimentCleanerLABELEDDataSMALLSAMPLE.csv')
print('The sample dataset has {} document and {} columns'.format(restaurant.shape[0], restaurant.shape[1]))
restaurant.head()

The sample dataset has 9 document and 6 columns


,sentiment,review,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,n,'Mike\'s Pizza High Point,NY Service was very slow and the quality was low. You would think they would know at least how to make good pizza,not. Stick to pre-made dishes like stuffed pasta or a salad. You should consider dining else where.',NaN,NaN
1,n,'i really like this buffet restaurant in Marshall street. they have a lot of selection of american,japanese,and chinese dishes. we also got a free drink and free refill. there are also different kinds of dessert. the staff is very friendly. it is also quite cheap compared with the other restaurant in syracuse area. i will definitely coming back here.',NaN,NaN
2,n,'After I went shopping with some of my friend,we went to DODO restaurant for dinner. I found worm in one of the dishes .',NaN,NaN,NaN
3,n,'Olive Oil Garden was very disappointing. I expect good food and good service (at least!!) when I go out to eat. The meal was cold when we got it,and the waitor had no manners whatsoever. Don\'t go to the Olive Oil Garden. ',NaN,NaN,NaN
4,n,'The Seven Heaven restaurant was never known for a superior service but what we experienced last week was a disaster. The waiter would not notice us until we asked him 4 times to bring us the menu. The food was not exceptional either. It took them though 2 minutes to bring us a check after they spotted we finished eating and are not ordering more. Well,never more. ',NaN,NaN,NaN


In [4]:
restaurant.isna().sum().sum()

19

Summary:
   * 9 instances
   * labels are located at fisrt column for each row
   * text data contain newline
   * back slash and apostrophe have to be removed
   

#### Strategies:
1. Fill NAs with spaces
2. forloop each rows and use string method to join each fo reviews
3. strip() method to get rid of leading and trailing spaces
4. rebuild a dataframe

In [5]:
# fill na with a space
restaurant = restaurant.fillna(' ')

In [6]:
def df_string_to_list(dataframe):
    '''
    expect: a dataframe with seperated strings in different columns 
    modify: fill the NAs with spaces, and combine the strings from each column for each row as a document
    return: a list of documents
    '''
    dataframe = dataframe.fillna(' ')
    # create two temp lists
    # second temp_list is for storing a list of string list for each document
    # final_list is for storing a list of strings for each document
    second_temp_list=[]
    final_list=[]
    for i, r in dataframe.iterrows():
        # create a temp list to store a list of strings for a document
        temp_list=[]
        for s in r:
            # append strings as a list of strings for a document
            temp_list.append(s)
        # append the string list for each documents
        second_temp_list.append(temp_list)
    # join the string list and remove trailing and leading spaces for each document    
    final_list = [' '.join(second_temp_list[i]).strip() for i in range(len(second_temp_list))]
    return final_list

In [7]:
review = df_string_to_list(restaurant.iloc[:,1:])

In [8]:
# create a new dataframe
restaurant_df = pd.DataFrame({'label': restaurant.sentiment, 'review':review})
# it is still messy in the strings
restaurant_df.head()

,label,review
0,n,'Mike\'s Pizza High Point NY Service was very slow and the quality was low. You would think they would know at least how to make good pizza not. Stick to pre-made dishes like stuffed pasta or a salad. You should consider dining else where.'
1,n,'i really like this buffet restaurant in Marshall street. they have a lot of selection of american japanese and chinese dishes. we also got a free drink and free refill. there are also different kinds of dessert. the staff is very friendly. it is also quite cheap compared with the other restaurant in syracuse area. i will definitely coming back here.'
2,n,'After I went shopping with some of my friend we went to DODO restaurant for dinner. I found worm in one of the dishes .'
3,n,'Olive Oil Garden was very disappointing. I expect good food and good service (at least!!) when I go out to eat. The meal was cold when we got it and the waitor had no manners whatsoever. Don\'t go to the Olive Oil Garden. '
4,n,'The Seven Heaven restaurant was never known for a superior service but what we experienced last week was a disaster. The waiter would not notice us until we asked him 4 times to bring us the menu. The food was not exceptional either. It took them though 2 minutes to bring us a check after they spotted we finished eating and are not ordering more. Well never more. '


In [9]:
# define functions for cleaning strings in dataframe
def clean_str(string):
    '''
    expect: an unclean string. ex: "'Mike\\'s Pizza High Point  NY Service was very slow and the quality was low.'"
    modify: remove new line and back slash; remove leading and trailing back slash and apostrophe
    return: a string
    '''
    import re
    temp = re.sub(r'\\n|\\','', string) # remove new line and back slash
    temp2 = temp.strip("'")       # remove leading and trailing apostrophe
    result = temp2.strip("\\")       # remove leading and trailing back slash
    return result

In [10]:
# clean the unwanted strings
restaurant_df['review'] = restaurant_df.review.map(clean_str)
restaurant_df['label']= restaurant_df.label.map({'n':'neg', 'p':'pos'})
restaurant_df.head()

,label,review
0,neg,Mike's Pizza High Point NY Service was very slow and the quality was low. You would think they would know at least how to make good pizza not. Stick to pre-made dishes like stuffed pasta or a salad. You should consider dining else where.
1,neg,i really like this buffet restaurant in Marshall street. they have a lot of selection of american japanese and chinese dishes. we also got a free drink and free refill. there are also different kinds of dessert. the staff is very friendly. it is also quite cheap compared with the other restaurant in syracuse area. i will definitely coming back here.
2,neg,After I went shopping with some of my friend we went to DODO restaurant for dinner. I found worm in one of the dishes .
3,neg,Olive Oil Garden was very disappointing. I expect good food and good service (at least!!) when I go out to eat. The meal was cold when we got it and the waitor had no manners whatsoever. Don't go to the Olive Oil Garden.
4,neg,The Seven Heaven restaurant was never known for a superior service but what we experienced last week was a disaster. The waiter would not notice us until we asked him 4 times to bring us the menu. The food was not exceptional either. It took them though 2 minutes to bring us a check after they spotted we finished eating and are not ordering more. Well never more.


In [11]:
restaurant_df.to_csv('restaurant_clean.csv')

## Dataset 2

In [12]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 60)

In [13]:
movie = pd.read_csv('MovieDataSAMPLE_labeledVERYSMALL.csv')
print('The sample dataset has {} document and {} columns'.format(movie.shape[0], movie.shape[1]))
movie.head()

The sample dataset has 5 document and 57 columns


,text,reviewclass,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56
0,'let\'s say you live at the end of an airport runway . \nlarge jetliners continuously pass over ...,rattling your windows and allowing few moments of peace . \nlet\'s say the ground beneath your ...,and high voltage power lines run across steel towers only yards from your home . \nnow let\'s s...,and offers you more than it\'s worth . \nwouldn\'t you jump in the air and have your bags packe...,and the house your castle ? \nthat\'s what they are to darryl kerrigan ( michael caton ),and although he\'s happy that a multi-national corporation would like to buy his home,sorry,it\'s not for sale . \nunfortunately,the corporation wields quite a bit of influence over the australian government,which serves darryl and his family with what amounts to an eviction notice . \nnot about to tak...,darryl enlists the help of his inept attorney friend dennis denuto ( tiriel mora ) and makes a ...,albeit pathetic,attempt to fight back . \nthe castle,an australian comedy by rob sitch,is perhaps one of the simplest movies i\'ve seen in recent memory,yet also one of the funniest . \nin fact,it is this film\'s reliance on simplicity which makes it so successful . \nthe good guys and th...,the humor is straightforward,and the storyline is as uncomplicated as it gets . \nit\'s the perfect setup for a relaxing,hilarious good time,and as much as i hate to admit it,i liked the fact that i didn\'t have to think all that much while watching this film . \ni just...,or big business,or whatever . \nwhat makes this film different is its characters . \ndarryl thinks he\'s got th...,he asks who would ever want to go out and eat in a fancy restaurant when he\'s got the meat loa...,and he swells with pride when his son dale ( stephen curry ) digs a hole in the yard . \nsteve ...,while wayne kerrigan is in jail,as a result of dennis denuto\'s incompetent courtroom maneuvers . \nthe great thing about the k...,but a whole lot of affection . \nscreenwriters sitch,santo cilauro,tom gleisner,and jane kennedy pit this lovable bunch of unlikely crusaders against a stoic court system and ...,further polarizing the extremes and deepening our feelings for the kerrigans . \nsympathy for t...,the laughter they bring us,and the family\'s wonderful characterization,simple as it may be,leaves us with a degree of emotional investment that causes us to actually care about what happ...,but trust me,it works . \ni particularly noted that the writers did not have to use slapstick in order to ge...,when executed well,can be genius . \nit\'s just that too often,a film will resort to it\'s use as a shortcut to amusing the audience,not always successfully . \ninstead,the jokes in the castle are all hinged upon the dialog uttered by these people who have no idea...,the are funny,and when they are with normal people,they are hilarious . \none of the few problems i saw with this film was a detour taken about mi...,it\'s a funny little detour that,while slightly distracting from the film\'s main drive,is still quite enjoyable . \nwith all the levity abound in the castle,the film still manages to take on a pretty heavy concept,that of the innate right of a citizen to keep his property,an issue which has been bandied about in courts across the civilized world with differing outco...,but it brings to the forefront the essence of the con

In [14]:
movie.isna().sum().sum()

81

Summary:
   * 5 instances
   * labels are located at last column for each row
   * text data contain newline
   * back slash and apostrophe have to be removed
    

#### Strategies:
1. a nested for loops to iterate each row
2. store review and label to lists
3. create pandas dataframe with two lists as dictionary format

In [15]:
review_list =df_string_to_list(movie)
label=[review_list[i][-3:] for i in range(len(review_list))]
review=[review_list[i][:-3].strip() for i in range(len(review_list))]

In [16]:
pd.set_option('display.max_colwidth', 1000)
# create a new dataframe
movie_df = pd.DataFrame({'label': label, 'review':review})
# it is still messy in the strings
movie_df.head()

,label,review
0,pos,'let\'s say you live at the end of an airport runway . \nlarge jetliners continuously pass over your house rattling your windows and allowing few moments of peace . \nlet\'s say the ground beneath your feet is also contaminated with toxic substances and high voltage power lines run across steel towers only yards from your home . \nnow let\'s say an airport conglomerate wants to buy your property in order to lengthen the runway and offers you more than it\'s worth . \nwouldn\'t you jump in the air and have your bags packed before you landed ? \nof course you would . \nbut what if this bit of land was your idea of paradise and the house your castle ? \nthat\'s what they are to darryl kerrigan ( michael caton ) and although he\'s happy that a multi-national corporation would like to buy his home sorry it\'s not for sale . \nunfortunately the corporation wields quite a bit of influence over the australian government which serves darryl and his family with what amounts...
1,pos,'in this good natured pleasent and easy going comedy bill murray ( ghostbusters 1984 ) plays grumpy weatherman phil conners who every year is sent to punxsutawney p . a to report on groundhog day . \nthe groundhog day ceremony involves a groundhog being lifted out of a box and if he dosen\'t see a shadow it will be an early spring . \nphil really hates the ceremony and not even his producer rita ( macdowell ) can change his mind . \nhowever fate has a cruel trick for phil and he starts re-living groundhog day over and over and over until he gradually likes it and rita falls in love with him . \ngroundhog day is a well written totatly unoffensive and funny comedy . \nthe screenwriters director ramis and danny rubin have written a funny warm but never overly senitmental comedy . \nalthough the idea of a day repeating over and over may sound tedious there are enough good jokes to hold the audiences attention throughout the whole film . \nr...
2,neg,"'i\'m really starting to wonder about alicia silverstone . \nsure she is one of the most beautiful creatures on god\'s green earth ( second only to that movie critic at large guy ) but when it comes to choosing what movies she stars in she always strikes out . \nthe crush was a slow-moving predictable piece of fluff . \nhideaway was a horrific novel adaptation alicia had only a minor role in . \nclueless was an annoying unfunny waste of time . \nand people have e-mailed me too saying clueless is a good movie and that i\'m the only one who doesn\'t like it . \none girl said if i\'d seen the movie with \"" an open mind \"" i would have enjoyed it . \nnothing could be further from the truth . \ni went into the theater expecting to love the movie . \nthe preview looked good and of course i\'m crazymadinlove with alicia but the movie was a bunch of bad jokes coming from whiny unlikable characters . \nalmost everyone i saw the movie with felt the same way . \nwhen we were..."
3,neg,'so what do you get when you mix together plot elements from various successful sci-fi films such as close encounters of the third kind 2001 : a space odyssey apollo 13 and contact ? \nwell whatever it is you\'d sure as hell hope that it would be a thousand times better than this shoddy attempt at such a melange considering the disastrous results we\'re left with here . \nthis is a film that takes a little bit of everything but ultimately adds up to a lot of nothing ! \nit\'s like i said . . . this \nmovie sucks . \nplot : a rescue crew of astronauts is sent down to mars in the year 2020 after an unknown energy force leads to a loss of contact with the previous gang of space aviators to visit the red planet . \ncritique : extremely underwhelming is the best way to describe this movie . \nuneven would be another . \nthe trailer for this movie actually showed some promise the buzz around it had been so-so and even the film itself starts off with a decent first twe...
4,neg,"'the law of crowd pleasing romantic movies states tha

In [17]:
movie_df.review = movie_df.review.map(clean_str)
movie_df.head()

,label,review
0,pos,let's say you live at the end of an airport runway . large jetliners continuously pass over your house rattling your windows and allowing few moments of peace . let's say the ground beneath your feet is also contaminated with toxic substances and high voltage power lines run across steel towers only yards from your home . now let's say an airport conglomerate wants to buy your property in order to lengthen the runway and offers you more than it's worth . wouldn't you jump in the air and have your bags packed before you landed ? of course you would . but what if this bit of land was your idea of paradise and the house your castle ? that's what they are to darryl kerrigan ( michael caton ) and although he's happy that a multi-national corporation would like to buy his home sorry it's not for sale . unfortunately the corporation wields quite a bit of influence over the australian government which serves darryl and his family with what amounts to an eviction notice . ...
1,pos,in this good natured pleasent and easy going comedy bill murray ( ghostbusters 1984 ) plays grumpy weatherman phil conners who every year is sent to punxsutawney p . a to report on groundhog day . the groundhog day ceremony involves a groundhog being lifted out of a box and if he dosen't see a shadow it will be an early spring . phil really hates the ceremony and not even his producer rita ( macdowell ) can change his mind . however fate has a cruel trick for phil and he starts re-living groundhog day over and over and over until he gradually likes it and rita falls in love with him . groundhog day is a well written totatly unoffensive and funny comedy . the screenwriters director ramis and danny rubin have written a funny warm but never overly senitmental comedy . although the idea of a day repeating over and over may sound tedious there are enough good jokes to hold the audiences attention throughout the whole film . ramis's direction...
2,neg,"i'm really starting to wonder about alicia silverstone . sure she is one of the most beautiful creatures on god's green earth ( second only to that movie critic at large guy ) but when it comes to choosing what movies she stars in she always strikes out . the crush was a slow-moving predictable piece of fluff . hideaway was a horrific novel adaptation alicia had only a minor role in . clueless was an annoying unfunny waste of time . and people have e-mailed me too saying clueless is a good movie and that i'm the only one who doesn't like it . one girl said if i'd seen the movie with "" an open mind "" i would have enjoyed it . nothing could be further from the truth . i went into the theater expecting to love the movie . the preview looked good and of course i'm crazymadinlove with alicia but the movie was a bunch of bad jokes coming from whiny unlikable characters . almost everyone i saw the movie with felt the same way . when we were walking out of the theater o..."
3,neg,so what do you get when you mix together plot elements from various successful sci-fi films such as close encounters of the third kind 2001 : a space odyssey apollo 13 and contact ? well whatever it is you'd sure as hell hope that it would be a thousand times better than this shoddy attempt at such a melange considering the disastrous results we're left with here . this is a film that takes a little bit of everything but ultimately adds up to a lot of nothing ! it's like i said . . . this movie sucks . plot : a rescue crew of astronauts is sent down to mars in the year 2020 after an unknown energy force leads to a loss of contact with the previous gang of space aviators to visit the red planet . critique : extremely underwhelming is the best way to describe this movie . uneven would be another . the trailer for this movie actually showed some promise the buzz around it had been so-so and even the film itself starts off with a decent first twenty minutes all le...
4,neg,"the law of crowd pleasing romantic movies states that the

In [18]:
movie_df.to_csv('movie_clean.csv')